<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [6]:
# get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2022-04-28 16:46:26--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  13.9MB/s    in 28s     

2022-04-28 16:46:54 (8.96 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [37]:
# 1 Import libraries

from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np

In [9]:
# 2 Setting parameters

num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100
batch_size_test = 100

In [11]:
# 3 Construct an ImageDataGenerator

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [12]:
# 4 Create a sequential model using Keras

model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers
model.layers[0].layers
model.layers[0].trainable = False
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
# 5 Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
#6 Fit the model

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 203s 674ms/step - loss: 0.0799 - accuracy: 0.9766 - val_loss: 0.0251 - val_accuracy: 0.9945
Epoch 2/2
301/301 [==============================] - 207s 686ms/step - loss: 0.0196 - accuracy: 0.9957 - val_loss: 0.0152 - val_accuracy: 0.9957


In [16]:
model.save('classifier_vgg_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [22]:
# 1 loading the ResNet50 saved model

from google.colab import drive
drive.mount('/content/drive')

model_resnet = load_model('/content/drive/MyDrive/Colab Notebooks/classifier_resnet_model.h5')
model_vgg = model

In [23]:
# 2 Construct  ImageDataGenerator

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_test,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [42]:
# 3 Evaluate ResNet50

score_resnet = model_resnet.evaluate_generator(generator=test_generator)
print(f'ResNet50 Loss = {np.round(score_resnet[0],4)} and Accuracy = {np.round(score_resnet[1],4)}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


ResNet50 Loss = 0.0016 and Accuracy = 1.0


In [43]:
# 3 Evaluate VGG16

score_vgg = model_vgg.evaluate_generator(generator=test_generator)
print(f'VGG16 Loss = {np.round(score_vgg[0],4)} and Accuracy = {np.round(score_vgg[1],4)}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


VGG16 Loss = 0.0132 and Accuracy = 0.996


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [31]:
# 1 Making predictions for both ResNet50 and VGG16

pred_resnet = model_resnet.predict_generator(generator=test_generator)
pred_vgg = model_vgg.predict_generator(generator=test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
# 2 Reporting the predictions for the first 5 images for each of ResNet50 and VGG16

def report_class(pred):
  '''
  Makes result and probability.
  Input:
    pred: np.array >>> prediction
  Returns:
    result: str >>> Positive/Negative
    proba: np.float32 >>> probability
  '''
  if pred[0] > pred[1]:
    result = 'Negative'
    proba = pred[0]
  else:
    result = 'Positive'
    proba = pred[1]

  return result, proba

print('Class predictions for ResNet50:\n')
for i in range(5):
  r, p = report_class(pred_resnet[i])
  print(f'{i + 1}: ', r, f' ({np.round(p * 100, 2)}%)')

print()

print('Class predictions for VGG16:\n')
for i in range(5):
  r, p = report_class(pred_vgg[i])
  print(f'{i + 1}: ', r, f' ({np.round(p * 100, 2)}%)')

Class predictions for ResNet50:

1:  Positive  (99.99%)
2:  Positive  (100.0%)
3:  Positive  (100.0%)
4:  Positive  (99.97%)
5:  Positive  (100.0%)

Class predictions for VGG16:

1:  Positive  (100.0%)
2:  Positive  (99.99%)
3:  Positive  (100.0%)
4:  Negative  (99.98%)
5:  Positive  (99.95%)


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).